In [362]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np
from random import sample
from collections import Counter

#df = pd.read_csv('data/data-from-emma/SmarterStayAlive.csv')
df = pd.read_csv('data/data-from-emma/Target.csv')

print(df.columns)

print(df.shape)

print(Counter(df.seed))

Index(['agent_name', 'seed', 't', 'action', 'missed_ball', 'xpos_ball',
       'ypos_ball', 'xpos_ball_prev', 'ypos_ball_prev', 'xpos_pad', 'ypos_pad',
       'xpos_pad_prev', 'ypos_pad_prev', 'board_alive', 'is_far_left',
       'is_far_right', 'score', 'pad_width', 'ball_speed', 'ball_down',
       'xdist_ball_pad', 'ydist_ball_pad', 'l2_ball_pad', 'num_bricks_left'],
      dtype='object')
(38241, 24)
Counter({9654690: 2000, 127525: 2000, 6816947: 2000, 6169021: 2000, 5509973: 2000, 596737: 2000, 9334909: 2000, 122168: 1816, 9989550: 1802, 3648516: 1752, 1116028: 1743, 3226525: 1702, 2247560: 1595, 4561254: 1588, 3736793: 1488, 9261983: 1338, 2241640: 1162, 9012387: 1067, 5082548: 1052, 3681492: 924, 7670649: 875, 2091658: 789, 6759291: 696, 1001231: 679, 6782754: 533, 3451451: 484, 693342: 401, 4045096: 323, 2401381: 293, 9583670: 139})


### The dataframe contains time steps for 30 different seeds


In [2]:
print(len(np.unique(df['seed'])))

30


#### Initially, we'll say "why did you go left?" (instead of 'right' or 'button1') is the outcome/query of interest

## Steps for training a model
- Calculate the outcome variable at every time point
- looping over all episodes, sample positive and negative data points
- discretize all variables

### We first have to do some variable construction and correction

- Convert the 'board_alive' variable to brick counts by column
- Correct 'ball_down' to correctly be False when the ball is maintaining its height

In [363]:
new_col_data = []
new_col_names = ["bricks_in_col_" + str(col).zfill(2) for col in range(0,18)]

for col in new_col_names:#add new (empty) columns to the dataframe
    df[col] = -1

for row in range(0,len(df.index)):
    board_alive = bin(int(df['board_alive'][row]))[2:].zfill(108)
    cols = [board_alive[(i*6):(i*6+6)] for i in range(0,18)]#pull out each column from the binary string (every 7 digits)
    col_counts = [sum([int(val) for val in list(col)]) for col in cols]#convert each string to a list of ints and sum them
    new_col_data.append(col_counts)

df[new_col_names] = new_col_data


In [364]:
dfs = []
for seed in np.unique(df.seed):
    df_seed = df[df.seed == seed].copy(deep=True)
    df_seed['ypos_ball_next'] = [list(df_seed.loc[df_seed['t'] == t+1,'ypos_ball'])[0] if (t+1) in df_seed['t'].values else float('nan') for t in df_seed.t]

    df_seed['ball_down'] = [False if list(df_seed[df_seed.t == t].ypos_ball == df_seed[df_seed.t == t].ypos_ball_next)[0] else list(df_seed[df_seed.t == t].ball_down)[0] for t in df_seed.t]
    dfs.append(df_seed)

df = pd.concat(dfs)
print(df)

      agent_name     seed     t   action  missed_ball   xpos_ball   ypos_ball  \
21652     Target   122168     1  button1        False  214.267949   81.000000   
21653     Target   122168     2  button1        False  212.535898   82.000000   
21654     Target   122168     3  button1        False  210.803848   83.000000   
21655     Target   122168     4  button1        False  209.071797   84.000000   
21656     Target   122168     5     left        False  207.339746   85.000000   
21657     Target   122168     6  button1        False  205.607695   86.000000   
21658     Target   122168     7  button1        False  203.875644   87.000000   
21659     Target   122168     8     left        False  202.143594   88.000000   
21660     Target   122168     9  button1        False  200.411543   89.000000   
21661     Target   122168    10  button1        False  198.679492   90.000000   
21662     Target   122168    11  button1        False  196.947441   91.000000   
21663     Target   122168   

In [5]:
df.iloc[3400]

agent_name                                    Target
seed                                          127525
t                                               1585
action                                         right
missed_ball                                    False
xpos_ball                                    160.862
ypos_ball                                    48.1418
xpos_ball_prev                               159.235
ypos_ball_prev                                51.796
xpos_pad                                         160
ypos_pad                                         143
xpos_pad_prev                                    160
ypos_pad_prev                                    143
board_alive         60877497971844372295859610308414
is_far_left                                    False
is_far_right                                   False
score                                            270
pad_width                                      small
ball_speed                                    

### Create variables from the column counts
- number of columns with 0, 1, or 2 bricks remaining
- number of columns left of the paddle w/1 or 2 bricks remaining
- number of bricks remaining left of the paddle
- number of bricks remaining right of the paddle

In [6]:
df['num_cols_0_bricks'] = [sum(df.iloc[row][new_col_names] == 0) for row in range(0,len(df.index))]

In [7]:
df['num_cols_1_brick'] = [sum(df.iloc[row][new_col_names] == 1) for row in range(0,len(df.index))]

In [8]:
df['num_cols_2_bricks'] = [sum(df.iloc[row][new_col_names] == 2) for row in range(0,len(df.index))]

In [9]:
#hard-coded column positions, taken from the json of a random Breakout game
col_positions = [i*12 for i in range(1,19)]

bricks_left_of_paddle = []
bricks_right_of_paddle = []
channels_left_of_paddle = []
channels_right_of_paddle = []
almost_channels_left_of_paddle = []
almost_channels_right_of_paddle = []

for row in range(0, len(df.index)):
    paddle_pos = df.iloc[row]['xpos_pad']
    cols_left_of_paddle = np.where(col_positions < paddle_pos)[0]#get which columns are left of paddle
    cols_left_of_paddle = [str(i).zfill(2) for i in cols_left_of_paddle]#convert into a padded string to match column names
    cols_right_of_paddle = np.where(col_positions > paddle_pos)[0]#get which columns are left of paddle
    cols_right_of_paddle = [str(i).zfill(2) for i in cols_right_of_paddle]#convert into a padded string to match column names
 
    prefix = new_col_names[0][0:(len(new_col_names[0])-2)]
    cols_left_of_paddle = [prefix + col for col in cols_left_of_paddle]#add the prefix to match column names
    cols_right_of_paddle = [prefix + col for col in cols_right_of_paddle]#add the prefix to match column names

    vals_left_of_paddle = df.iloc[row][cols_left_of_paddle]
    vals_right_of_paddle = df.iloc[row][cols_right_of_paddle]
    bricks_left_of_paddle.append(sum(vals_left_of_paddle))
    bricks_right_of_paddle.append(sum(vals_right_of_paddle))
    
    channels_left_of_paddle.append(sum(vals_left_of_paddle == 0))
    channels_right_of_paddle.append(sum(vals_right_of_paddle == 0))

    almost_channels_left_of_paddle.append(sum([(val in [1,2]) for val in vals_left_of_paddle]))
    almost_channels_right_of_paddle.append(sum([(val in [1,2]) for val in vals_right_of_paddle]))


df['bricks_left_of_paddle'] = bricks_left_of_paddle
df['bricks_right_of_paddle'] = bricks_right_of_paddle
df['channels_left_of_paddle'] = channels_left_of_paddle
df['channels_right_of_paddle'] = channels_right_of_paddle
df['almost_channels_left_of_paddle'] = almost_channels_left_of_paddle
df['almost_channels_right_of_paddle'] = almost_channels_right_of_paddle

In [10]:
df.iloc[3400]

agent_name                                                   Target
seed                                                         127525
t                                                              1585
action                                                        right
missed_ball                                                   False
xpos_ball                                                   160.862
ypos_ball                                                   48.1418
xpos_ball_prev                                              159.235
ypos_ball_prev                                               51.796
xpos_pad                                                        160
ypos_pad                                                        143
xpos_pad_prev                                                   160
ypos_pad_prev                                                   143
board_alive                        60877497971844372295859610308414
is_far_left                                     

## We have to do some data cleaning - some predictors have NaN fields
- remove any rows that have no action
- for xpos_ball_prev and ypos_ball_prev at t=0, set to xpos_ball and ypos_ball
- same for xpos_pad_prev and ypos_pad_prev

In [377]:
has_action = [not pd.isnull(val) for val in df['action']]
df = df[has_action]

nans = np.isnan(df['xpos_ball_prev'])
df.loc[nans,'xpos_ball_prev'] = df.loc[nans, 'xpos_ball']
df.loc[nans,'ypos_ball_prev'] = df.loc[nans, 'ypos_ball']

nans = np.isnan(df['xpos_pad_prev'])
df.loc[nans,'xpos_pad_prev'] = df.loc[nans, 'xpos_pad']
df.loc[nans,'ypos_pad_prev'] = df.loc[nans, 'ypos_pad']


### Define outcome variable, current options are:
- Why did you move left?
- Why did you aim left?
  - only defined at the point when the paddle hits the ball (use dummy value otherwise)
  - locate every time step where the ball hits the paddle
  - compare the relative position of the ball at its 'hit' time step to its position at the following time step
    - (if following time step, ball's x position < current x position, outcome = TRUE, otherwise outcome = FALSE)
    
### Along with outcome variable, need to specifiy 'outcome_indicator'  This determines which time points are valid with respect to the outcome (for example, for "why did you aim left?", outcome_indicator = 'ball_hit'.  This should be a column of the dataframe that = True for valid time points
- If the value for outcome_indicator doesn't already exist (such as "ball_hit"), it should be defined in the relevant outcome-specific if-statement below.
- If any time point is valid for a given outcome (for example, for "why did you move left?"), outcome_indicator should be NaN

In [365]:
outcome = 'aimed_left'
outcome_indicator = 'ball_hit'

#outcome = 'moved_left'
#outcome_indicator = float('nan')

#why did you move left?
if outcome == 'move_left':
    df['outcome'] = (df['action'] == 'left')

    
#why did you aim left?
if outcome == 'aimed_left':
    dfs = []
    #break it down by seed, to make sure we can look at the next time step easily
    for seed in np.unique(df['seed']):
        df_seed = df[df['seed'] == seed].copy(deep=True)
        df_seed['ball_down_prev'] = [list(df_seed.loc[df_seed['t'] == t-1,'ball_down'])[0] if (t-1) in df_seed['t'].values else float('nan') for t in df_seed['t']]
        df_seed['ball_hit'] = [(df_seed.loc[row,'ball_down_prev'] == True) & (df_seed.loc[row, 'ball_down'] == False) & (df_seed.loc[row, 'ypos_ball'] > 130) for row in df_seed.index]

        df_seed['aimed_left'] = [((list(df_seed[df_seed.t == t]['xpos_ball'])[0] > list(df_seed[df_seed.t == t+2]['xpos_ball'])[0]) & list(df_seed[df_seed.t == t]['ball_hit'])[0]) if (t+2) in df_seed.t.values else False for t in df_seed.t]
        df_seed['aimed_right'] = [((list(df_seed[df_seed.t == t]['xpos_ball'])[0] < list(df_seed[df_seed.t == t+2]['xpos_ball'])[0]) & list(df_seed[df_seed.t == t]['ball_hit'])[0]) if (t+2) in df_seed.t.values else False for t in df_seed.t]

        dfs.append(df_seed)

    df = pd.concat(dfs)

    df['outcome'] = df['aimed_left']
    
if pd.isnull(outcome_indicator):
    df['outcome_indicator'] = True
else:
    df['outcome_indicator'] = df[outcome_indicator]

In [359]:
seed = df.seed[0]
df_seed = df[df.seed == seed]

#print(df_seed[df_seed.ypos_ball > 137])
print(df_seed[((df_seed.t > 1945) & (df_seed.t < 1955)) | ((df_seed.t > 50) & (df_seed.t < 65))].ball_down)
#min(df.xdist_ball_pad)
Counter(df.outcome == True)

50       True
51       True
52       True
53       True
54       True
55       True
56       True
57       True
58       True
59       True
60       True
61      False
62      False
63      False
1945     True
1946     True
1947     True
1948     True
1949     True
1950     True
1951     True
1952    False
1953    False
Name: ball_down, dtype: object


AttributeError: 'DataFrame' object has no attribute 'outcome'

## Important note about valid predictors
- some predictors aren't valid for certain outcomes.  For "Why did you aim left", for example, ypos_ball is NOT a valid predictor.  Because we'll only consider data points where the ball is close to the paddle, there's essentially no variance in ypos_ball, and intervening to change it changes the validity of the outcome.  Currently, this shows up by breaking the covariate models - it's never seen any value outside a very narrow range.

In [379]:
predictors = ['missed_ball', 'xpos_ball', 'xpos_ball_prev', 'xpos_pad',
        'xpos_pad_prev', 'is_far_left', 'is_far_right', 'score', 'pad_width', 'ball_speed', 'ball_down',
        'xdist_ball_pad', 'ydist_ball_pad', 'l2_ball_pad', 'num_bricks_left'], 'num_cols_0_bricks', 'num_cols_1_brick',
        'num_cols_2_bricks', 'bricks_right_of_paddle', 'bricks_left_of_paddle', 'channels_left_of_paddle',
        'channels_right_of_paddle', 'almost_channels_left_of_paddle', 'almost_channels_right_of_paddle']

#### Discretize all predictors (outcome is already boolean)
#### Convert booleans to 0,1

In [368]:
df_stable = df.copy(deep=True)

In [380]:
for predictor in predictors:
    print(predictor)
    if isinstance(df[predictor][0], (np.bool_, bool)):
        print(predictor + " is boolean")
        #convert booleans to 0/1 rather than 'True'/'False'
        df[predictor] = [1 if val == 'True' else 0 for val in df[predictor]]
    if isinstance(df[predictor][0], str):
        if len(np.unique(df[predictor])) > 2:
            print("warning: " + predictor + " should be converted into numbers")
        else:#don't discretize, just convert to integer values, since order is irrelevant if there are only two possible values
            df[predictor] = [1 if val == df[predictor][0] else 0 for val in df[predictor]]
    
    

missed_ball
xpos_ball
xpos_ball_prev
xpos_pad
xpos_pad_prev
is_far_left
is_far_right
score
pad_width
ball_speed
ball_down
xdist_ball_pad
ydist_ball_pad
l2_ball_pad
num_bricks_left


#### For each seed, get all positive data points and sample an equal number of negative data points

num_pos_samples_per_episode is how many positive samples ot take from every episode
num_neg_samples_per_episode is how many negative samples to take from every episode

the time points sampled represent the covariates time point - we'll have to get the outcome frome time_lag steps later

Make sure the sampled time points aren't within time_lag of the beginning of an episode (so we can sample the covariates from before that)

importantly, after this point, everything except the column 'outcome' referes to time t - time_lag.

For example, with time_lag = 4, the row with t = 100 contains observations from t = 96 and outcome from t = 100.  This means, even if outcome is whether or the agent aimed left, 'outcome' and 'aimed_left' may not match. (since 'aimed_left' is calculated at time 96, not time 100)

In [381]:
num_pos_samples_per_episode = 10
num_neg_samples_per_episode = 10
time_lag = 5

samples = []
for seed in np.unique(df['seed']):
    df_seed = df[df.seed == seed]
    positive_samples_t = df_seed[df_seed.outcome & (df_seed.t >= time_lag)].t.values - time_lag
    positive_samples_t = sample(list(positive_samples_t), min(num_pos_samples_per_episode, len(positive_samples_t)))
    positive_samples = df_seed[[t in positive_samples_t for t in df_seed.t]]
    positive_samples = positive_samples.assign(outcome = True)
    
    negative_samples_t = df_seed[~df_seed.outcome & df_seed.outcome_indicator & (df_seed.t >= time_lag)].t.values - time_lag
    negative_samples_t = sample(list(negative_samples_t), min(num_neg_samples_per_episode, len(negative_samples_t)))
    negative_samples = df_seed[[t in negative_samples_t for t in df_seed.t]]
    negative_samples = negative_samples.assign(outcome = False)

    samples.append(positive_samples)
    samples.append(negative_samples)
samples = pd.concat(samples)

print(samples.shape)


(317, 49)


### Learn model of outcome given covariates

In [382]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1000)

rf.fit(samples[predictors], samples['outcome'])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### In order to answer counterfactual queries, we're going to need to discretize each predictor
### For consistency, then, we'll discretize now, and use those discretized bins for the joint model

This code doesn't actually apply the discretization scheme to any of the data.  It just calculates the discretization intervals and saves them in discretization_dict

In [383]:
num_discretization_bins = 5

discretization_dict = dict()
for predictor in predictors:
    if isinstance(df[predictor][0], float) | len(pd.unique(df[predictor])) > 8:
        print("Discretizing " + predictor)
        not_discretized_yet = True
        curr_num_bins = 1
        
        while not_discretized_yet:
            try:
                if curr_num_bins == 1:#we can't do a single bin, so give up on density-based binning
                    print("Giving up on discretizing " + predictor + " using density-based binning")
                    discretization = list(pd.cut(df[predictor], bins=num_discretization_bins))
                else:
                    discretization = list(pd.qcut(df[predictor], q=curr_num_bins))
                discretization_dict[predictor] = sorted(np.unique(discretization))#save the discretization scheme

                #set the endpoints to -inf and inf
    #            for i in range(0, len(discretization)):
    #                if discretization[i] == discretization_dict[predictor][0]:#set lower endpoint to 0
    #                    discretization[i] = pd.Interval(-float("inf"), discretization[i].right)
    #                if discretization[i] == discretization_dict[covar][len(discretization_dict[predictor])-1]:
    #                    discretization[i] = pd.Interval(discretization[i].left, float("inf"))

                discretization_dict[predictor] = sorted(np.unique(discretization))
                print(discretization_dict[predictor])
                
                not_discretized_yet = False
                print("Discretizing " + predictor + " into " + str(len(np.unique(discretization))) + " bins\n")

            except ValueError:
                curr_num_bins -= 1

Discretizing xpos_ball
Giving up on discretizing xpos_ball using density-based binning
[Interval(12.201, 55.51, closed='right'), Interval(55.51, 98.603, closed='right'), Interval(98.603, 141.696, closed='right'), Interval(141.696, 184.79, closed='right'), Interval(184.79, 227.883, closed='right')]
Discretizing xpos_ball into 5 bins

Discretizing xpos_ball_prev
Giving up on discretizing xpos_ball_prev using density-based binning
[Interval(12.201, 55.51, closed='right'), Interval(55.51, 98.603, closed='right'), Interval(98.603, 141.696, closed='right'), Interval(141.696, 184.79, closed='right'), Interval(184.79, 227.883, closed='right')]
Discretizing xpos_ball_prev into 5 bins

Discretizing xpos_pad
Giving up on discretizing xpos_pad using density-based binning
[Interval(-0.24, 48.0, closed='right'), Interval(48.0, 96.0, closed='right'), Interval(96.0, 144.0, closed='right'), Interval(144.0, 192.0, closed='right'), Interval(192.0, 240.0, closed='right')]
Discretizing xpos_pad into 5 bins

### Learn joint model of covariates, in the form of separate predictive random forests for each covariate

In [384]:
#preds = rf.predict(df[predictors])
rf_dict = dict()
for covar in predictors:
    print(covar)
    if covar in discretization_dict.keys():#need to apply discretization scheme to the current covar
        discretization = discretization_dict[covar]
        val_to_discretize = samples[covar]
        val_discretized = []
        for val in val_to_discretize:
            is_disc_value = [val in disc for disc in discretization]#figure out which discretization category the value is in
            val_discretized.append(discretization[np.where(is_disc_value)[0][0]])#get the actual interval corresponding to that location
        curr_outcome = [str(val) for val in val_discretized]
    else:
        curr_outcome = samples[covar]
    rf_dict[covar] = RandomForestClassifier(n_estimators=100)
    rf_dict[covar].fit(samples[predictors].loc[:, samples[predictors].columns != covar], curr_outcome)

missed_ball
xpos_ball
xpos_ball_prev
xpos_pad
xpos_pad_prev
is_far_left
is_far_right
score
pad_width
ball_speed
ball_down
xdist_ball_pad
ydist_ball_pad
l2_ball_pad
num_bricks_left


In [387]:
query = df.loc[[32720]]
covar_rf = rf_dict['xpos_ball']

model_preds = covar_rf.predict_proba(query[predictors].loc[:, query[predictors].columns != 'xpos_ball'])[0]
model_preds

array([0.  , 0.89, 0.01, 0.01, 0.09])

### Now that we've learned all the relevant models, we can finally provide counterfactual explanations!
- take in a query time point (must satisfy outcome_indicator)
- apply the time lag
- iterate over all possible (discretized) values for all predictors
- for each value, check if the predicted outcome crosses over the .5 threshold to give a different prediction
- if so, this is an explanation - report the likelihood of this explanation using the corresponding covariate model#

In [361]:
query = df.loc[sample(list(df[df.outcome_indicator].index),1)]
#print(query)
#query = df.loc[[32720]]
query_tick = list(query.t)[0]
query_seed = list(query.seed)[0]
observed_outcome = list(query.outcome)[0]

if query_tick - time_lag < 0:
    print("ERROR - invalid query time (too early in episode)")

#apply the time lag
query = df[(df.t == (query_tick - time_lag)) & (df.seed == query_seed)]
query.outcome = observed_outcome


possible_explanations = pd.DataFrame(columns=['variable', 'from_value', 'to_value', 'relative_prob'])
pred_outcome = rf.predict(query[predictors])[0]
if pred_outcome != observed_outcome:
    print("Unexpected outcome - model prediction differs from observation")
    
for var in predictors:
    observed_val = query[var].iloc[0]
#    print("observed: " + str(observed_val))
    if var in discretization_dict.keys():
#        print(var + " was discretized!")
        possible_ranges = discretization_dict[var].copy()
        observed_val = possible_ranges[np.where([(observed_val in curr_range) for curr_range in possible_ranges])[0][0]]
        possible_ranges.remove(observed_val)
        #sample from the ranges to get actual values for intervention
        possible_vals = [np.random.uniform(curr_range.left, curr_range.right) for curr_range in possible_ranges]
    else:
#        print(var + " was not discretized!")
        possible_vals = list(np.unique(df[var]))
        possible_vals.remove(observed_val)
    
    #loop over the possible different values for var, and see if they change the predicted outcome
    vals_with_different_outcome = []
    for val in possible_vals:
        intervened_covars = query.copy(deep=True)
        intervened_covars[var] = val
        intervened_outcome = rf.predict(intervened_covars[predictors])
        outcome_probs = rf.predict_proba(intervened_covars[predictors])
        
        if intervened_outcome != observed_outcome:
            vals_with_different_outcome.append(val)
            
    if len(vals_with_different_outcome) == 0:
        print("No interventions found for " + str(var))
    else:
        print("These settings of " + str(var) + " change the outcome: " + str(vals_with_different_outcome))
        
    #Now apply the covariate model to get the relative probabilities of these new values for var 
    covar_rf = rf_dict[var]
    covar_classes = [str(curr_class) for curr_class in list(covar_rf.classes_)]
    for val in vals_with_different_outcome:
        if var in discretization_dict.keys():
            val = possible_ranges[np.where([(val in curr_range) for curr_range in possible_ranges])[0][0]]
        model_preds = covar_rf.predict_proba(query[predictors].loc[:, query[predictors].columns != var])[0]
#        print(model_preds)
#        print("observed_val = " + str(observed_val))
#        print("intervened_val = " + str(val))
        observed_prob = model_preds[covar_classes.index(str(observed_val))]

        if ~(val in covar_classes):
            intervened_prob = 0
        else:
            intervened_prob = model_preds[covar_classes.index(str(val))]
        
        if observed_prob == 0:
            change_prob = 0
        else:
            change_prob = intervened_prob/observed_prob
        possible_explanations.loc[len(possible_explanations)] = [var, observed_val, val, change_prob]

print(possible_explanations)

AttributeError: 'DataFrame' object has no attribute 'outcome_indicator'